In [8]:
try:
    import tangelo
    import pennylane as qml
except ModuleNotFoundError:
    !pip install git+https://github.com/goodchemistryco/Tangelo.git@develop  --quiet
    !pip install pyscf
    !pip install qulacs qiskit qiskit-aer  
    !pip install pennylane --upgrade



In [9]:
from tangelo import SecondQuantizedMolecule

LiH = [('Li', (0, 0, 0)),('H', (0, 0, 1.5949))] #these triples refer to coordinates
# the 1.5949 is experimentally determined to be the space betwn the elements
#see here for example https://cccbdb.nist.gov/expgeom2x.asp?casno=1333740
#use frozen_orbitals = None to set everything active for calculations
mol_LiH = SecondQuantizedMolecule(LiH, q=0, spin=0, basis="sto-3g")

# triples can be made using spin #, abs diff between alpha and beta electrons
#alpha = spin up, beta = spin-down
#spin = alpha - beta electrons, 2 electrons make alpha(it seems) beta is single 
#electron occupying an orbital
mol_LiH_t = SecondQuantizedMolecule(LiH, q=0, spin=2, basis="sto-3g")


In [10]:
#Recieve visualization of the molecule for verification

#there should be 6 spin orbitals, which can hold a max of 2 electrons
#2 are occupied, with one active and one frozen and should have 4
#total electrons

# Note, frozen orbitals mean that some electrons are treated as fixed
# so if one orbital is frozen => only 2 electrons get qbits as opposed to 
#others in the frozen orbital

#mo_occ parameter tells use where our electrons are!
print(mol_LiH) # 2, 2 split between 0 and 1 oribtals
print(mol_LiH_t) # 2,1,1 splits between 0
print(mol_LiH.n_mos, mol_LiH.n_sos)

SecondQuantizedMolecule(xyz=[('Li', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 1.5949))], q=0, spin=0, solver=<tangelo.toolboxes.molecular_computation.integral_solver_pyscf.IntegralSolverPySCF object at 0x7c18df66a590>, n_atoms=2, n_electrons=4, basis='sto-3g', ecp={}, symmetry=False, uhf=False, mf_energy=-7.862026959394131, mo_energies=array([-2.34864417, -0.28570476,  0.07826185,  0.16393842,  0.16393842,
        0.54912925]), mo_occ=array([2., 2., 0., 0., 0., 0.]), n_mos=6, n_sos=12, active_occupied=[1], frozen_occupied=[0], active_virtual=[2, 3, 4, 5], frozen_virtual=[])
SecondQuantizedMolecule(xyz=[('Li', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 1.5949))], q=0, spin=2, solver=<tangelo.toolboxes.molecular_computation.integral_solver_pyscf.IntegralSolverPySCF object at 0x7c18df66a2d0>, n_atoms=2, n_electrons=4, basis='sto-3g', ecp={}, symmetry=False, uhf=False, mf_energy=-7.766133365335707, mo_energies=NPArrayWithTag([-2.34705371, -0.26589778,  0.01337911,  0.16088823,
                 0.16088823

In [12]:
from tangelo.algorithms import FCISolver, CCSDSolver
#classical energy calculations using FCI and CCSD
#FCI = bad scale good acc, CCSD = good scale worse ACC
def getClassicalGround(mol):
    fci_solver = FCISolver(mol)
    fci_energy = fci_solver.simulate()
    
    
    
    ccsd_solver = CCSDSolver(mol)
    ccsd_energy = ccsd_solver.simulate()
    return fci_energy, ccsd_energy
    
fci_energy,ccsd_energy = getClassicalGround(mol_LiH)
print(f"LiH FCI Groud State Energy: {fci_energy}")
print(f"CCSD Groud State Energy: {ccsd_energy}")
print(f"Percent difference of FCI and CCSD: {100*(fci_energy-ccsd_energy)/fci_energy}%")

LiH FCI Groud State Energy: -7.882175990801265
CCSD Groud State Energy: -7.882175990941336
Percent difference of FCI and CCSD: -1.7770608778679271e-09%


In [4]:
#Implement a VQE Energy solver
from tangelo.algorithms import VQESolver

#100 cycles for speed's sake
def getVqeEnergies(mol):
    vqe_options = {"molecule": mol, "verbose": True}
    
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    vqe_energy = vqe_solver.simulate()

    return vqe_energy
    
#print(f"VQE Ground State Energy: {vqe_energy}")
#print(f"Percent difference of FCI and VQE: {100*(fci_energy-vqe_energy)/fci_energy}%")
"""
VQE Ground State Energy: -7.882390470334529
Percent difference of FCI and VQE: 0.00016416314034363796%
"""

'\nVQE Ground State Energy: -7.882390470334529\nPercent difference of FCI and VQE: 0.00016416314034363796%\n'

In [8]:
from pprint import pprint
pprint(vars(vqe_solver))

{'ansatz': <tangelo.toolboxes.ansatz_generator.uccsd.UCCSD object at 0x7d916cd85410>,
 'ansatz_options': {},
 'backend': <tangelo.linq.target.target_qulacs.QulacsSimulator object at 0x7d916c420210>,
 'backend_options': {'n_shots': None, 'noise_model': None, 'target': None},
 'builtin_ansatze': {<BuiltInAnsatze.UCCSD: <class 'tangelo.toolboxes.ansatz_generator.uccsd.UCCSD'>>,
                     <BuiltInAnsatze.UCCGD: <class 'tangelo.toolboxes.ansatz_generator.uccgd.UCCGD'>>,
                     <BuiltInAnsatze.UpCCGSD: <class 'tangelo.toolboxes.ansatz_generator.upccgsd.UpCCGSD'>>,
                     <BuiltInAnsatze.VSQS: <class 'tangelo.toolboxes.ansatz_generator.vsqs.VSQS'>>,
                     <BuiltInAnsatze.UCC1: <tangelo.toolboxes.ansatz_generator.rucc.RUCC object at 0x7d91745bef90>>,
                     <BuiltInAnsatze.HEA: <class 'tangelo.toolboxes.ansatz_generator.hea.HEA'>>,
                     <BuiltInAnsatze.ILC: <class 'tangelo.toolboxes.ansatz_generator.ilc.ILC'>>,

In [5]:
#tangelo implements trotter suzuki time evolution
#we can use that module to do things, but we need to find
#the qubit hamiltonian of our molecule
#Steps: get hamiltonian, build Trotter circuit, simulate circuit in diff ways
#(noisy, unnoisy, quantum machine hardware, etc.)
from tangelo.toolboxes.unitary_generator.trotter_suzuki import TrotterSuzukiUnitary

NUM_STEPS = 5

LiH_gnd_hamiltonian = vqe_solver.qubit_hamiltonian
ts_model = TrotterSuzukiUnitary(qubit_hamiltonian=LiH_gnd_hamiltonian)
ts_circuit = ts_model.build_circuit(NUM_STEPS)

In [6]:
#we now need to do a simulation of our trotter suzuki circuit
print(ts_circuit)
ts_circuit.draw()


Circuit object. Size 3965 

RZ        target : [0]   parameter : 9.580279999453282
RX        target : [0]   parameter : 1.5707963267948966
RX        target : [2]   parameter : 1.5707963267948966
CNOT      target : [1]   control : [0]   
CNOT      target : [2]   control : [1]   
RZ        target : [2]   parameter : 12.530077206294187
CNOT      target : [2]   control : [1]   
CNOT      target : [1]   control : [0]   
RX        target : [2]   parameter : -1.5707963267948966
RX        target : [0]   parameter : -1.5707963267948966
H         target : [0]   
H         target : [2]   
CNOT      target : [1]   control : [0]   
CNOT      target : [2]   control : [1]   
RZ        target : [2]   parameter : 12.530077206294187
CNOT      target : [2]   control : [1]   
CNOT      target : [1]   control : [0]   
H         target : [2]   
H         target : [0]   
RX        target : [0]   parameter : 1.5707963267948966
RX        target : [8]   parameter : 1.5707963267948966
CNOT      target : [1]   co

In [29]:
#Try to have better implementation that trotter suzuki using QDrift
#Qubit Hamiltonian is a Qubit Operator from openFermion
import numpy
from pprint import pprint
numpy.set_printoptions(threshold=10_000)

LiH_qml_hamiltonian = qml.qchem.import_operator(LiH_gnd_hamiltonian)

#some initial state can be used as input using hadamard as sub like in test
dev = qml.device("default.qubit", wires=10)
@qml.qnode(dev)
def buildQDrift(H):    
    qml.Hadamard(0)
    qml.QDrift(H, 1, n=5) # n == number of expoenentiaed terms
    return qml.probs()
pprint(buildQDrift(LiH_qml_hamiltonian))


tensor([0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. 